In [42]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.colors import SymLogNorm
import scienceplots

import astropy.units as u
from astropy import constants
from astropy.table import Table
from astropy.coordinates import SkyCoord,FK5,ICRS
from astropy.io import fits,ascii
from astropy.wcs import WCS, utils as wcsutils
from astropy.stats import sigma_clipped_stats
import astropy.visualization as vis
from astropy.modeling import models, fitting
from astropy.nddata import StdDevUncertainty, Cutout2D
from astropy.convolution import Gaussian2DKernel,Gaussian1DKernel
from astropy.cosmology import Planck18
from scipy.signal import find_peaks
#from scipy.constants import c



from photutils.aperture import EllipticalAperture, SkyEllipticalAperture

import os, sys, glob, pdb, scipy

from importlib import reload
import pyregion as pyreg
from spectral_cube import SpectralCube
from reproject import reproject_interp
import regions
from regions.shapes.circle import CircleSkyRegion,CirclePixelRegion
from regions import CircleAnnulusSkyRegion, CircleAnnulusPixelRegion, RectangleSkyRegion
from regions import PixCoord
sys.path.append('/disk/bifrost/yuanze/KBSS/MUSEQSO/scripts')
sys.path.append('/disk/bifrost/yuanze/KBSS/CubEx_run/scripts')
import run_cubetools_v1 as ctools
import makeMask_MUSE as mmask
reload(ctools)
reload(mmask)
sys.path.append('/disk/bifrost/yuanze/software/GalfitS/src')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import images as IM
import sed_interp
reload(IM)

brightu = r"Brightness $\rm (10^{-4}~erg~s^{-1}~arcsec^{-2}~cm^{-2}~\AA^{-1})$"
def imshow_astro(img, wcsinfo = None, figsize = (10,10), colorbar =True,
               cblabel="", cbfrac = 0.035, norm = None,
               stretch = vis.LinearStretch(), cmap = "hot",
               vrange = (None, None)):
    
    _, med, std = sigma_clipped_stats(img.data)
    
    fig =  plt.figure(figsize = figsize)
    if wcsinfo:
        ax = plt.subplot(projection = wcsinfo)
    else:
        ax = plt.subplot()
    
    vmin, vmax = vrange
    
    if not vmin:
        vmin = med
    if not vmax:
        vmax = med + 10*std
    if not norm:
        norm = vis.ImageNormalize(vmin = vmin, vmax = vmax, stretch = stretch)
    im = ax.imshow(img, norm = norm, cmap = cmap)
    if colorbar:
        cb = plt.colorbar(im, label = cblabel, fraction =cbfrac)
    return fig, ax, im
KBSSpath="/disk/bifrost/yuanze/KBSS"
source_table = ascii.read(KBSSpath+"/KCWI/KBSS_faint_AGN.list",format="ipac")

#MUSEQSO= ascii.read(KBSSpath+"/KCWI/MUSEQSO_machine_readable_updated2.list",format="ipac")
#museqso_path = os.path.join(KBSSpath, "MUSEQSO")
#votable_file = os.path.join("fixed_file.xml")

In [65]:
all_data_dir

['/disk/bifrost/yuanze/KBSS/SDSSQSO/J2309+1127/kcwir',
 '/disk/bifrost/yuanze/KBSS/SDSSQSO/J2118+0253/kcwir',
 '/disk/bifrost/yuanze/KBSS/SDSSQSO/J0037+1414/kcwir']

In [62]:

reload(mmask)
from datetime import datetime# Making white-light cutout
root_directory = KBSSpath#,"table['Name']=='AGN_2223'"
filters = ["table['KCWI'] == 'yes'","table['Field']=='SDSSQSO'","table['Name']!='Lab5'","table['Name']!='FSzP1170'"]
dtype="KBSS"
#condition=[qsos_bright['Field']!="Q1623",qsos_bright['Field']!="Q0142"]
#source_table = qsos_bright#[(qsos_bright['Field']!="Q1623") & (qsos_bright['Field']!="Q0142")]

#print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)
#radius=6
dovar=True
MakeQSOwhite=False
channel="red"
postfix=""
all_directories,qsotab,all_data_dir = ctools.find_directories_from_ascii(source_table, root_directory,filters=filters,KBSS=(dtype=="KBSS"),channel=channel)

if channel=="red":
    postfix="-red"

for ne,element in enumerate(qsotab):#source_table[qsos_bright['Field']=="Q1623"]:
    #print(element)
    field = element["Field"]
    Qtype=element["Type"]
    objname=element["Name"]
    cname=element["CName"]
    subpath=all_directories[ne]
    sub_data_path=all_data_dir[ne]
    #CubRunpath=KBSSpath+"/CubEx_run/"+field
    if MakeQSOwhite:
        fn=sub_data_path+"/{}-{}{}_icubes_wcs.fits".format(field,cname,postfix)
    else:
        if Qtype < 1.9:
            fn=subpath+"/{}-{}{}_icubes.PSFSub.fits".format(field,objname,postfix)
        else:
            fn=subpath+"/{}-{}{}_icubes.CONTSub.fits".format(field,objname,postfix)
    try:
        hdu0=fits.open(fn)
    except FileNotFoundError:
        print(f"No Red channel data for {field}-{objname}")
        continue
    hdr = hdu0[0].header
    wc = (np.arange(hdr['NAXIS3']) - hdr['CRPIX3'] + 1) * hdr['CD3_3'] + hdr['CRVAL3']
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt").astype(np.int64)
    zmask = mmask.determine_mask(wc,[f"continuum_{channel}"], element['zlya'],channel=channel)
    xpix = element["x"]
    ypix = element["y"]
    if len(zmask[0])==0:
        if channel == "red":
            zmask = mmask.determine_mask(wc,["CIII]"], element['zlya'],channel=channel)
        else:
            zmask = mmask.determine_mask(wc,["NV"], element['zlya'],channel=channel)
        wimage = np.nansum(hdu0[0].data[zmask[0][0]+100:zmask[1][0]+120,:],axis=0)
    else:
        wimage = np.nansum(hdu0[0].data[zmask[0][0]:zmask[1][0],:],axis=0)
    wimage[wimage==0] = np.nan
    header = fits.Header()
    header['DATE'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    header['CREATOR'] = 'Yuanze Ding'
    header['COMMENT'] = 'White-light cutout from 3D data cube'
    header['YSOURCE'] = ypix
    header['XSOURCE'] = xpix
    header['OBJECT'] = objname
    # Extract WCS information from the original header
    wcs_keys = ['CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'CRPIX1', 'CRPIX2', 'CDELT1', 'CDELT2', 'CUNIT1', 'CUNIT2', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2']
    for key in wcs_keys:
        if key in hdr:
            header[key] = hdr[key]

    hdu = fits.PrimaryHDU(wimage,header=header)
    if MakeQSOwhite:
        output_fn = subpath+"/{}-{}{}_icubes.white.fits".format(field,objname,postfix)
    else:
        if Qtype < 1.9:
            output_fn = subpath+"/{}-{}{}_icubes.PSFSub.white.fits".format(field,objname,postfix)
        else:
            output_fn = subpath+"/{}-{}{}_icubes.CONTSub.white.fits".format(field,objname,postfix)
    hdulist = fits.HDUList([hdu])
    print("writing:",output_fn)
    hdulist.writeto(output_fn, overwrite=True)
    if dovar:
        fnvar=sub_data_path+"/{}-{}{}_vcubes.fits".format(field,cname,postfix)
        hdu0_var=fits.open(fnvar)
        #if len(zmask[0])==0:
          #  wvar = np.nansum(hdu0_var[0].data[zmask[0][0]:zmask[1][0],:],axis=0)
        #else:
        wvar = np.nansum(hdu0_var[0].data[zmask[0][0]:zmask[1][0],:],axis=0)
        wvar[wvar==0] = np.nan
        output_fnvar = subpath+"/{}-{}{}_icubes.whitevar.fits".format(field,objname,postfix)
        hduvar = fits.PrimaryHDU(wvar,header=header)
        hdulist = fits.HDUList([hduvar])
        hdulist.writeto(output_fnvar, overwrite=True)
        print("writing:",output_fnvar)
            #hdu2=fits.open(KBSSpath+"/MUSEQSO/Q-0347-383/ADP.2016-06-02T13:21:57.103.fits")

Line: CIII], Min index: 129, Max index: 170
writing: /disk/bifrost/yuanze/KBSS/SDSSQSO/J2309+1127/kcwir/SDSSQSO-J2309+1127-red_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/SDSSQSO/J2309+1127/kcwir/SDSSQSO-J2309+1127-red_icubes.whitevar.fits
Line: continuum_red, Min index: 984, Max index: 1055
writing: /disk/bifrost/yuanze/KBSS/SDSSQSO/J2118+0253/kcwir/SDSSQSO-J2118+0253-red_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/SDSSQSO/J2118+0253/kcwir/SDSSQSO-J2118+0253-red_icubes.whitevar.fits
Line: continuum_red, Min index: 452, Max index: 518
writing: /disk/bifrost/yuanze/KBSS/SDSSQSO/J0037+1414/kcwir/SDSSQSO-J0037+1414-red_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/SDSSQSO/J0037+1414/kcwir/SDSSQSO-J0037+1414-red_icubes.whitevar.fits


In [63]:
import warnings
from regions import Regions
reload(ctools)
root_directory = KBSSpath
#filters = ["table['KCWI'] == 'yes'","table['Type'] < 1.5"]
#filters = ["table['KCWI'] == 'yes'","table['Name']!='Lab5'","table['Name']!='FSzP1170'"]

dtype="KBSS"
#condition=[qsos_bright['Field']!="Q1623",qsos_bright['Field']!="Q0142"]
#channel="red"
all_directories,qsotab,all_data_dir = ctools.find_directories_from_ascii(source_table, root_directory,filters=filters,KBSS=(dtype=="KBSS"),channel=channel)
if channel=="red":
    postfix="-red"
else:
    postfix=""
print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)
#radius=6
cutr=20
mark_region=False
for ne,element in enumerate(qsotab):
    #qname = element["Name"]
    field = element["Field"]
    
    
    objname=element["Name"]
    cname=element["CName"]
    Qtype=element["Type"]
    print(objname)
    subpath=all_directories[ne]
    CubRunpath=subpath
    ra = element['RA']
    dec = element['Decl']
    sc= SkyCoord(ra,dec,unit=(u.deg,u.deg))
   # qname="LBQS 2139-4434"
    if Qtype < 1.9:
        fname=subpath+"/{}-{}{}_icubes.PSFSub.white.fits".format(field,objname,postfix)
    else:
        fname=subpath+"/{}-{}{}_icubes.CONTSub.white.fits".format(field,objname,postfix)
    fnamevar=subpath+"/{}-{}{}_icubes.whitevar.fits".format(field,objname,postfix)
    try:
        hduvar=fits.open(fnamevar)
    except FileNotFoundError:
        print(f"No Red channel data for {field}-{objname}")
        continue
    var=hduvar[0].data
    #var[var==0]=np.nan
    mean_sky_var, _, _ = sigma_clipped_stats(var, sigma=2.0, maxiters=None)
    img = IM.image(fname,hdu=0,unit=u.Jy)
    xpix = element["x"]
    ypix = element["y"]
    wcs=WCS(img.data.header)
    
    artists=None
    metas=None
    if mark_region:
        artists=[]
        metas=[]
        if os.path.exists(KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg"):
            reg= ctools.read_regions(KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg",pix_scale=wcs.pixel_scale_matrix[1,1]*3600)
            for r in reg:
                artists.append(r.to_pixel(wcs).as_artist())
                meta=dict(r.meta)
                meta["fontsize"]=5
                metas.append(meta)
        else:
            print("warning: no region file found")
    center = [ypix,xpix]
    imcut, cp = img.img_cut(sc.ra.value,sc.dec.value,cutr)
    sky_mean, sky_median, sky_std = sigma_clipped_stats(imcut, sigma=3.0, maxiters=None)
    print("sky level from vcube:",np.sqrt(mean_sky_var),"; from sigma clipping:",sky_std,"; bkg level:",sky_mean)
    img.sources_skycord_cut = [xpix,ypix]
    #img.cut_mask_image = np.zeros_like(img.cut_sigma_image)
    sdmask = ctools.create_circular_mask(img.data.shape, center, 9)
    try:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            img.cut_mask_image = img.generate_cutmask(4,nsigma=2,sky_level=np.sqrt(mean_sky_var),sdmask=sdmask,data = img.data.data,deblend=False,addgrow=1,nlevels=32,source_dia=5,contrast=0.001)
    except:
        for warning in w:
            print(f"Warning: {warning.message}")
        continue
    ctools.maskplot(img.data.data,img.cut_mask_image,meta=metas,artist=artists,output=root_directory+"/diagnostic/"+field+"-"+objname+postfix+"_maskmap.pdf",center=center)
    if Qtype < 1.9:
        outname = subpath+"/{}-{}{}_icubes.PSFSub.mask.fits".format(field,objname,postfix)
    else:
        outname = subpath+"/{}-{}{}_icubes.CONTSub.mask.fits".format(field,objname,postfix)
    ctools.write_fits_cube(img.cut_mask_image.astype(np.int32),img.data.header,outname)
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt",dtype=int)

Number of directories found: 3
J2309+1127
347.28145922 11.45361696
50.5 50.4999999999579
sky level from vcube: 0.035734016794115184 ; from sigma clipping: 0.018652752 ; bkg level: 0.0010219553


J2118+0253
319.60054518 2.89629169
50.5 50.4999999999579
sky level from vcube: 0.010543758852358646 ; from sigma clipping: 0.014168688 ; bkg level: 0.00033128064


J0037+1414
9.35195761 14.24909923
50.5 50.4999999999579
sky level from vcube: 0.029655738293751777 ; from sigma clipping: 0.024127059 ; bkg level: -0.0024439637


In [49]:
img.generate_cutmask(4,nsigma=2,sky_level=0.5*np.sqrt(mean_sky_var),sdmask=sdmask,data = img.data.data,deblend=False,addgrow=1,nlevels=32,source_dia=5,contrast=0.001)

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [53]:
import re
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, PixCoord
from astropy import units as u

# Read the region file
file_path = KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg"
with open(file_path, 'r') as file:
    lines = file.readlines()

# Regex to extract information
pattern = re.compile(r'fk5;circle\((.*?),(.*?),(.*?)i\) #color=(.*?)  width=2 text={(.*?)}')

regions = []

for line in lines:
    match = pattern.match(line)
    if match:
        ra, dec, radius, color, text = match.groups()
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame='fk5')
        radius = float(radius) * u.arcsec  # assuming the radius is in arcseconds
        
        # Create a CircleSkyRegion
        region = CircleSkyRegion(center=coord, radius=radius)
        regions.append(region)

# Now `regions` contains all the parsed CircleSkyRegions
# You can manipulate them or plot them using regions library

# Example: Printing region details
for region in regions:
    print(region)

Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79699292, 13.27175306)>
radius: 3.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80051, 13.2711325)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80004083, 13.271705)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80209958, 13.27299472)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79884458, 13.27309694)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79855333, 13.27198778)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79724583, 13.27124722)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equino